In [ ]:
import os
from shutil import copy
import random
import torch.nn as nn
import torch
from torch.nn import functional as F

from PIL import Image

import matplotlib.pyplot as plt
import json

import torchvision
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
import torch.optim as optim
from tqdm import tqdm
import shutil

import random
from sklearn.metrics import classification_report
import torchvision.datasets as datasets
import cv2
import numpy as np
from PIL import ImageDraw
import pandas as pd
import json

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve,f1_score, precision_score,recall_score

Construct and train the models:

In [ ]:
## VGG model

class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=False):
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))  # add an adaptive average pooling layer
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(512*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes)
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        # N x 3 x 224 x 224
        x = self.features(x)
        # N x 512 x 7 x 7
        x = self.avgpool(x)  #perform adaptive averaging pooling
        # N x 512 x 1 x 1
        x = x.view(x.size(0), -1)
        # N x 512*7*7
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # He initialization
                nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                # He initialization
                nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)


def make_features(cfg: list):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def vgg(model_name="vgg16", **kwargs):
    try:
        cfg = cfgs[model_name]
    except:
        print("Warning: model number {} not in cfgs dict!".format(model_name))
        exit(-1)
    model = VGG(make_features(cfg), **kwargs)
    return model

In [ ]:
## ResNet model

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        nn.init.kaiming_normal_(self.conv1.weight, mode='fan_out', nonlinearity='relu')
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, blocks, stride):
        strides = [stride] + [1]*(blocks-1)
        layers = []
        for stride in strides:
            layers.append(ResBlock(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
model = ResNet(num_classes=2)
print(model)

In [ ]:
net = vgg()
print(net)

In [ ]:
## Function for training

def train(model_name):
    global train_losses, train_accuracies, val_losses, val_accuracies, val_f1_scores, val_precision_scores, val_recall_scores
    global fpr, tpr, roc_auc, precision, recall
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using {} device.".format(device))

    data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
        "val": transforms.Compose([transforms.Resize((224, 224)),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    }

    image_path = '/content/drive/MyDrive/Deepfashion_data/outdata'
    assert os.path.exists(image_path), "{} path does not exist.".format(image_path)

    train_dataset = datasets.ImageFolder(root=os.path.join(image_path, "train"), transform=data_transform["train"])
    train_num = len(train_dataset)

    class_list = train_dataset.class_to_idx
    class_dict = dict((val, key) for key, val in class_list.items())
    json_str = json.dumps(class_dict, indent=4)
    with open('/content/drive/MyDrive/model/class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    batch_size = 32
    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])
    print('Using {} dataloader workers every process'.format(nw))

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=nw)

    validate_dataset = datasets.ImageFolder(root=os.path.join(image_path, "val"), transform=data_transform["val"])
    val_num = len(validate_dataset)
    validate_loader = torch.utils.data.DataLoader(validate_dataset, batch_size=batch_size, shuffle=False, num_workers=nw)
    print("Using {} images for training, {} images for validation.".format(train_num, val_num))
    lr = 0.0001

    epochs = 30
    best_acc = 0.0
    net = vgg(model_name=model_name, num_classes=2, init_weights=True)
    net.to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=lr)

    save_path = '/content/drive/MyDrive/model/{}_7:3.pth'.format(model_name)
    train_steps = len(train_loader)

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    val_f1_scores = []
    val_precision_scores = []
    val_recall_scores = []

    for epoch in range(epochs):
        # Train the model
        net.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0
        total_f1_score = 0
        total_auc = 0
        total_accuracy = 0

        train_bar = tqdm(train_loader)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            outputs = net(images.to(device))
            loss = loss_function(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # Calculate the training accuracy
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels.to(device)).sum().item()
            train_acc = train_correct / train_total

            # Print statistics
            running_loss += loss.item()
            train_bar.desc = "Train Epoch [{}/{}] Loss: {:.3f} Acc: {:.3f}".format(epoch + 1, epochs, loss, train_acc)

        # Validate the model
        net.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_true_labels = []
        val_pred_scores = []

        with torch.no_grad():
            for val_data in validate_loader:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))

                # Record the result
                val_true_labels.extend(val_labels.numpy())
                val_pred_scores.extend(torch.softmax(outputs, dim=1).cpu().numpy()[:, 1])

                loss = loss_function(outputs, val_labels.to(device))
                val_loss += loss.item()

                _, predicted = outputs.max(1)
                val_total += val_labels.size(0)
                val_correct += predicted.eq(val_labels.to(device)).sum().item()

            val_acc = val_correct / val_total
            val_loss /= len(validate_loader)

        # Calculate the indicators
        val_pred_labels = np.array(val_pred_scores) > 0.5
        val_f1_score = f1_score(val_true_labels, val_pred_labels)
        val_precision_score = precision_score(val_true_labels, val_pred_labels)
        val_recall_score = recall_score(val_true_labels, val_pred_labels)

        # Record the loss and accuracy
        train_losses.append(running_loss / train_steps)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        val_f1_scores.append(val_f1_score)
        val_precision_scores.append(val_precision_score)
        val_recall_scores.append(val_recall_score)

        print('[Epoch %d] Train Loss: %.3f Train Accuracy: %.3f Val Loss: %.3f Val Accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, train_acc, val_loss, val_acc))
        print('Val F1 Score: {:.3f}, Precision Score: {:.3f}, Recall Score: {:.3f}'.format(val_f1_score,
                                                                                            val_precision_score,
                                                                                            val_recall_score))

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), save_path)

    fpr, tpr, _ = roc_curve(val_true_labels, val_pred_scores)
    roc_auc = auc(fpr, tpr)
    precision, recall, _ = precision_recall_curve(val_true_labels, val_pred_scores)

    # Convert a NumPy array to a nested Python list
    fpr_lst = fpr.tolist()
    tpr_lst = tpr.tolist()
    # roc_auc_lst = roc_auc.tolist()
    precision_lst = precision.tolist()
    recall_lst = recall.tolist()

    # Encode a Python list as a string in JSON format
    fpr_str = json.dumps(fpr_lst)
    tpr_str = json.dumps(tpr_lst)
    # roc_auc_str = json.dumps(roc_auc_str)
    precision_str = json.dumps(precision_lst)
    recall_str = json.dumps(recall_lst)

    global result_dict
    result_dict = {'train_accuracy': train_accuracies, 'val_accuracy': val_accuracies,
                   'val_f1_score': val_f1_scores, 'val_precision_score': val_precision_scores,
                   'val_recall_score': val_recall_scores, 'fpr': fpr_str, 'tpr': tpr_str, 'roc_auc': roc_auc,
                   'precision': precision_str, 'recall': recall_str}
    save_result_path = '/content/drive/MyDrive/model/result_' + model_name + '.json'
    with open(save_result_path, 'w') as f:
        json.dump(result_dict, f)

    print('Finished Training')

In [ ]:
## Tain the model
model_name = "vgg16"
train(model_name)

Discover the unknown unknowns：

In [ ]:
# Define subclasses of class ImageFolder
class CustomImageFolder(datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super().__init__(root=root, transform=transform, target_transform=target_transform)

# Define class unknownData and blindData
class UnknownData:
    def __init__(self, image, true_label, predicted_label, confidence, img_tensor,  img_path):
        self.image = image
        self.true_label = true_label
        self.predicted_label = predicted_label
        self.confidence = confidence
        self.img_tensor = img_tensor.unsqueeze(0)
        self.img_path = img_path


class BlindData:
    def __init__(self, image, true_label, predicted_label, confidence, img_tensor,  img_path):
        self.image = image
        self.true_label = true_label
        self.predicted_label = predicted_label
        self.confidence = confidence
        self.img_tensor = img_tensor.unsqueeze(0)
        self.img_path = img_path

In [ ]:
## Function for evaluating instances

def evaluate(model, validate_loader, device, over_confidence, samples, show_wrong=False, show_image=False):
    model.eval()
    val_correct = 0
    val_total = 0
    confidences = []

    unknown_num = 0
    blind_num = 0
    unknown_data = []
    blind_data = []

    sample_indices = random.sample(list(range(len(validate_loader.dataset))), samples)
    for i, index in enumerate(sample_indices):
        val_images, val_labels = validate_loader.dataset[index]
        val_images = val_images.unsqueeze(0).to(device) # add batch dimension

        img_np = val_images[0].detach().cpu().permute(1, 2, 0).numpy()
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        img_np = (img_np * std) + mean
        img_np = np.clip(img_np, 0, 1)
        img_np = (img_np * 255.0).astype(np.uint8)
        img = Image.fromarray(img_np).convert('RGB')

        img_tensor = val_images.squeeze(0).detach().cpu()
        val_labels = torch.tensor([val_labels]).to(device)
        out = model(val_images)
        _, predicted = torch.max(out.data, 1)
        confidence = torch.softmax(out, dim=1)[0][predicted[0]].item()

        # Determine if an unknown unknown
        if predicted[0] != val_labels[0]:
            unknown_num += 1
            if validate_loader.dataset.classes[val_labels[0]] == 'dress':
                img_name = '/content/drive/MyDrive/Deepfashion_data/unknown_data/dress/unknown_{}_true_{}_pred_{}_conf_{:.4f}.jpg'.format(unknown_num,
                                  validate_loader.dataset.classes[val_labels[0]],   validate_loader.dataset.classes[predicted[0].item()], confidence)
            else:
                img_name = '/content/drive/MyDrive/Deepfashion_data/unknown_data/shorts/unknown_{}_true_{}_pred_{}_conf_{:.4f}.jpg'.format(unknown_num,
                                  validate_loader.dataset.classes[val_labels[0]],   validate_loader.dataset.classes[predicted[0].item()], confidence)

            unknown_data.append(UnknownData(val_images, val_labels, predicted, confidence, img_tensor, img_name))
            img.save(img_name)
            if confidence >= over_confidence:
                blind_num += 1

                if validate_loader.dataset.classes[val_labels[0]] == 'dress':
                    img_name = '/content/drive/MyDrive/Deepfashion_data/blind_data/dress/blind_{}_true_{}_pred_{}_conf_{:.4f}.jpg'.format(unknown_num,
                                      validate_loader.dataset.classes[val_labels[0]],   validate_loader.dataset.classes[predicted[0].item()], confidence)
                else:
                    img_name = '/content/drive/MyDrive/Deepfashion_data/blind_data/shorts/blind_{}_true_{}_pred_{}_conf_{:.4f}.jpg'.format(unknown_num,
                                      validate_loader.dataset.classes[val_labels[0]],   validate_loader.dataset.classes[predicted[0].item()], confidence)

                blind_data.append(BlindData(val_images, val_labels, predicted, confidence, img_tensor, img_name))
                img.save(img_name)


        if show_wrong and predicted[0] == val_labels[0]:
            continue

        # Show image of unknown unknown
        img = None
        if show_image:
            img_np = val_images[0].detach().cpu().permute(1, 2, 0).numpy()
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img_np = (img_np * std + mean) * 255.0
            img_np = np.clip(img_np, 0, 255)
            img_np = img_np.astype(np.uint8)
            img = Image.fromarray(img_np)
            true_label = validate_loader.dataset.classes[val_labels[0]]
            pred_label = validate_loader.dataset.classes[predicted[0].item()]
            plt.title('True label: {}  \nPredicted label: {}  \nConfidence: {:.2f}%'
              .format(true_label, pred_label, confidence*100))
            plt.imshow(img)
            plt.axis('off')
            plt.show()

        val_correct += predicted.eq(val_labels).sum().item()
        val_total += val_labels.size(0)
        confidences.append(confidence)

    # save unknown data and blind data to files
    torch.save([ud.__dict__ for ud in unknown_data], 'unknown_data.pth')
    torch.save([bd.__dict__ for bd in blind_data], 'blind_data.pth')

    val_acc = val_correct / val_total
    return unknown_num, blind_num, blind_data, unknown_data

In [ ]:
batch_size = 32
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])

data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
        "val": transforms.Compose([transforms.Resize((224, 224)),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

image_path = '/content/drive/MyDrive/Deepfashion_data/outdata'

# Using the defined subclass CustomImageFolder, the dataset attribute is needed in evaluate function.
validate_dataset = CustomImageFolder(root=os.path.join(image_path, "val"),
                                       transform=data_transform["val"])
validate_loader = torch.utils.data.DataLoader(validate_dataset, batch_size=batch_size, shuffle=False, num_workers=nw)

In [ ]:
# Same parameters and network structure as the previously trained model
model = vgg(model_name="vgg16", num_classes=2, init_weights=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
#model.eval()

# Load the trained model
saved_model_path = "/content/drive/MyDrive/model/vgg16_7:3.pth"
model.load_state_dict(torch.load(saved_model_path, map_location=device))

In [ ]:
## Display unknown unknowns information

unknown_num, blind_num, blind_data, unknown_data = evaluate(model, validate_loader, device, over_confidence=0.65, samples=800, show_wrong=True, show_image=True)

Interprete the unknown unknowns with CAM:

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
## CAM method
def draw_CAM(model, img_dir, save_dir, transform=None, visual_heatmap=False):

    for classes in os.listdir(img_dir):
        img_folder_path = os.path.join(img_dir, classes)
        save_folder_path = os.path.join(save_dir, classes)
        for i in os.listdir(img_folder_path):
            img_path = os.path.join(img_folder_path, i)
            print(img_path)
            save_path = os.path.join(save_folder_path, i)
            # save_path = "CAM_" + save_path
            print(save_path)

            # Image loading & pre-processing
            img = Image.open(img_path).convert('RGB')
            if transform:
                img = transform(img)
            else:
                transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
                img = transform(img)

            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            img = img.unsqueeze(0).to(device)


            # features is the feature map output by conv13
            model.eval()
            features = model.features(img)
            x = features.view(features.size(0), -1)
            output = model.classifier(x)

            # Auxiliary function to read the intermediate gradient
            def extract(g):
                global features_grad
                features_grad = g

            # The output of the highest scoring category
            pred = torch.argmax(output).item()
            pred_class = output[:, pred]

            # Get the gradient
            features.register_hook(extract)
            pred_class.backward()
            grads = features_grad

            # Adaptive average pooling
            pooled_grads = torch.nn.functional.adaptive_avg_pool2d(grads, (1, 1))

            # Remove batch_size
            pooled_grads = pooled_grads[0]
            features = features[0]

            # 512 is the number of channels in the last layer of the feature
            # Matrix multiplication
            for i in range(512):
                features[i, ...] *= pooled_grads[i, ...]
                # features[0] *= pooled_grads[0]
                # features[1] *= pooled_grads[1]
                # ……
                # features[511] *= pooled_grads[511]

            heatmap = features.cpu().detach()
            heatmap = np.mean(heatmap.numpy(), axis=0)

            # Positive pixels are retained, negative values are set to 0
            heatmap = np.maximum(heatmap, 0)
            heatmap /= np.max(heatmap)

            # if visual_heatmap:
            #     plt.matshow(heatmap)
            #     plt.show()

            img = cv2.imread(img_path)  # load original image
            print(img.shape[1])

            # Resize the heat map to the same size as the original image
            heatmap_SIZE = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
            # Convert heat maps to RGB format
            heatmap_R = np.uint8(255 * heatmap_SIZE)

            # Apply heat maps to raw images
            heatmap_RGB = cv2.applyColorMap(heatmap_R, cv2.COLORMAP_JET)
            # Convert cv plots to plt plots, otherwise the thermal part will turn blue
            heatmap_RGB = cv2.cvtColor(heatmap_RGB, cv2.COLOR_BGR2RGB)

            alpha = 0.4  # heat map transparency parameter
            x, y, z = np.where(heatmap_RGB > 0)
            heatmap_only = np.zeros_like(img)
            heatmap_only[x, y, :] = heatmap_RGB[x, y, :]

            # Fuse images to increase contrast and saturation
            superimposed_img = heatmap_only * alpha + img * (1 - alpha)
            superimposed_img = superimposed_img.astype(np.uint8)

            fig, axs = plt.subplots(2, 2, figsize=(10, 10))

            img = Image.open(img_path).convert('RGB')

            axs[0, 0].imshow(heatmap)
            axs[0, 0].set_title('(a)heatmap')

            axs[0, 1].imshow(heatmap_RGB)
            axs[0, 1].set_title('(b)RGB heatmap')

            axs[1, 0].imshow(img)
            axs[1, 0].set_title('(c)image')

            axs[1, 1].imshow(superimposed_img)
            axs[1, 1].set_title('(d)image with CAM')


            plt.rcParams['font.size'] = 14

            plt.tight_layout()

            plt.show()

            plt.savefig(save_path)

In [ ]:
save_dir = '/content/drive/MyDrive/Deepfashion_data/test_data'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

img_dir = '/content/drive/MyDrive/Deepfashion_data/test_data'
draw_CAM(model, img_dir, save_dir, transform=None, visual_heatmap=True)

Interprete the unknown unknowns with LIME:

In [ ]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple lime

In [ ]:
import lime
from lime import lime_image

In [ ]:
from skimage.segmentation import mark_boundaries
import skimage

In [ ]:
## Define the classifier function predict

def predict(images):
    input_tensors = []
    for image in images:
        # Perform image preprocessing and convert it to PyTorch tensor
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        input_tensor = preprocess(Image.fromarray(image)).unsqueeze(0)
        input_tensors.append(input_tensor)

    # Put the input tensor into the model for forward propagation
    with torch.no_grad():
        outputs = [model(input_tensor) for input_tensor in input_tensors]

    # Calculate the prediction (return the output vector without taking the one with the highest probability)
    predicted_labels = torch.cat(outputs, dim=0)

    return predicted_labels.numpy()

In [ ]:
# LIME method

def limeToshow(image_list, topK, lime_save_dic, num):

    global explanation_list
    explanation_array = []
    i = num
    for image_path in image_list:
        # Load an image file and convert it to a Numpy array
        i += 1
        print(image_path)
        lime_save_path = lime_save_dic + str(i)
        print(lime_save_path)

        pil_img = Image.open(image_path).convert('RGB')
        img_array = np.array(pil_img)
        img_array = np.expand_dims(img_array, axis=0)

        # Create an interpreter object
        explainer = lime_image.LimeImageExplainer()
        explanation = explainer.explain_instance(img_array[0], classifier_fn=predict, top_labels=2)
        explanation_list.append(explanation)

        # visualization
        temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=topK, hide_rest=False)
        temp = (temp - temp.min()) / (temp.max() - temp.min())

        plt.imshow(mark_boundaries(temp, mask))
        plt.savefig(lime_save_path)

In [ ]:
image_list = []

dir_path = '/content/drive/MyDrive/Deepfashion_data/blind_data/'

for file in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, file)):
        image_list.append(file)

In [ ]:
topK = 2
lime_save_dic = '/content/drive/MyDrive/Deepfashion_data/lime_data/'

limeToshow(image_list, topK, lime_save_dic, len(image_list))